In [ ]:
!nvidia-smi

Sun Mar  7 15:01:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Colab如何加装Google drive 硬盘：https://colab.research.google.com/notebooks/io.ipynb#scrollTo=XDg9OBaYqRMd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/'Shared drives'/JXNU_C/HXM/IPC_to_CLC/

/content/drive/Shared drives/JXNU_C/HXM/IPC_to_CLC


In [ ]:
!pwd

/content/drive/Shared drives/JXNU_C/HXM/IPC_to_CLC


In [ ]:
!ls

Bilinear_CNN.ipynb
Bi_LSTM.ipynb
data_KFold_1
drive-download-20210306T124752Z-001.zip
error_log
ipc_to_clc_data_and_result_类目相似度_结果备份
log
other_network
output
sentence_transformers
si_model.ipynb
test_sentence_transformers.ipynb
Text_CNN.ipynb
train_sentence_transformers.ipynb
Transformer_Encoder.ipynb


In [ ]:
!pip install ipdb transformers sentence-transformers

## 基于sentence_transformers实现预训练语言模型孪生网络，并进行训练。

In [ ]:
import os
import math
import logging
import ipdb
import torch

from torch import nn
from torch.utils.data import DataLoader
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, InputExample, LoggingHandler, SentenceTransformer, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import *
from datetime import datetime
from tqdm import tqdm

#设置随机种子，保证实验的可重复性
seed = 42

torch.manual_seed(seed)  # 为CPU设置随机种子
torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
#如果使用多个GPU，应该使用torch.cuda.manual_seed_all()为所有的GPU设置种子

#用以保证模型的可重复性，实验的可重复性
torch.backends.cudnn.deterministic = True

#设置日志格式
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])


#模型名字  distiluse-base-multilingual-cased-v2 
model_name = "bert-base-chinese"
#是否自定义模型(与模型名字有关)
custom_model = True

batch_size = 32
num_epochs = 8
weight_decay = 0.01

# 损失函数类型： sample_ContrastiveLoss   sample_MultipleNegativesRankingLoss   sample_TripletLoss
loss_type = "sample_ContrastiveLoss"

# 五折交叉验证，第几折选择: 0 1 2 3 4
kfold = "4"

#训练好的模型保存路径
model_save_path = "output/" + str(kfold) + "-" + str(num_epochs) + "-" + "training_ipc-to-clc_" + model_name + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

if not os.path.exists(model_save_path):
  os.makedirs(model_save_path)

#判断GPU是否可用
device = "cuda" if torch.cuda.is_available() else 'cpu'



#自定义模型，包括transformers中的模型也可以用(可以自定义模型框架)
if custom_model:
  # Use model_name for mapping tokens to embeddings
  word_embedding_model = models.Transformer(model_name, model_args={'output_hidden_states': True})

  transformerEncoder_model = models.TransformerEncoder(word_embedding_model.get_word_embedding_dimension(), nhead=8, num_layers=2, dropout=0.1)

  # Apply mean pooling to get one fixed sized sentence vector
  pooling_model = models.Pooling(transformerEncoder_model.get_word_embedding_dimension(),
                              pooling_mode_mean_tokens=True,
                              pooling_mode_cls_token=False,
                              pooling_mode_max_tokens=False)

  model = SentenceTransformer(modules=[word_embedding_model, transformerEncoder_model, pooling_model], device=device)

#加载sentence_transformers已经训练过的模型
else:
  model = SentenceTransformer(model_name, device=device)


#train文件路径
train_dataset_path = "data_KFold_1/" + loss_type + "/data_sample_" + kfold + "/ipc_clc_train.txt"

#valid文件路径(这个只有 sample_MultipleNegativesRankingLoss 需要，因为训练数据中没有label，需要在训练前将验证数据从训练集中分离出来)
valid_dataset_path = "data_KFold_1/" + loss_type + "/data_sample_" + kfold + "/ipc_clc_valid.txt"


#读入训练数据
logging.info("读入训练数据...")

train_shuliang = 0

train_samples = []
if loss_type == "sample_MultipleNegativesRankingLoss":
  with open(train_dataset_path, 'r', encoding='utf8') as fIn:
    for row in fIn:
      train_shuliang += 1
      row = row.strip()
      row = row.split('\t')
      train_samples.append(InputExample(texts=[row[0], row[1]])) # MultipleNegativesRankingLoss 无label
else:
  with open(train_dataset_path, 'r', encoding='utf8') as fIn:
      train_valid_number = 0
      for row in fIn:
        train_valid_number = train_valid_number + 1
        if train_valid_number % 5 == 0:
          continue
        else:
          train_shuliang += 1
          row = row.strip()
          row = row.split('\t')
          if loss_type == "sample_ContrastiveLoss":
            train_samples.append(InputExample(texts=[row[0], row[1]], label=int(row[2]))) # ContrastiveLoss 有label
          else:
            train_samples.append(InputExample(texts=[row[0], row[1], row[2]])) # TripletLoss 无label

print('训练集大小：' + str(train_shuliang))        

#构造训练数据 DataLoader
train_data = SentencesDataset(train_samples, model=model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)


#定义训练的损失函数
if loss_type == "sample_ContrastiveLoss":
  train_loss = losses.OnlineContrastiveLoss(model=model, margin=0.5)  #有label
elif loss_type == "sample_MultipleNegativesRankingLoss":
  train_loss = losses.MultipleNegativesRankingLoss(model=model)  #无label
else:
  train_loss = losses.TripletLoss(model=model, triplet_margin=1)  #无label 默认triplet_margin=5




#读入验证集数据
logging.info("读入验证集数据...")

valid_shuliang = 0

valid_samples = []
#sample_MultipleNegativesRankingLoss直接从文件中读取，其他loss的验证集将直接从训练数据中得到，1/5的训练集数据将作为验证集
#需要注意的是，如果只有一种label，则evaluator无法进行验证，所以是二分类的要保证验证集中两种类别都有
if loss_type == "sample_MultipleNegativesRankingLoss":
  with open(valid_dataset_path, 'r', encoding='utf8') as fIn:
    for row in fIn:
      valid_shuliang += 1
      row = row.strip()
      row = row.split('\t')
      valid_samples.append(InputExample(texts=[row[0], row[1]], label=int(row[2]))) #验证集需要 label
else:   
  with open(train_dataset_path, 'r', encoding='utf8') as fIn:
    train_valid_number = 0
    for row in fIn:
      train_valid_number = train_valid_number + 1
      if train_valid_number % 5 == 0:
        valid_shuliang += 1
        row = row.strip()
        row = row.split('\t')
        if loss_type == "sample_ContrastiveLoss":
          valid_samples.append(InputExample(texts=[row[0], row[1]], label=int(row[2]))) #验证集需要 label
        else:
          valid_samples.append(InputExample(texts=[row[0], row[1]], label=int(1))) # TripletLoss pos
          valid_samples.append(InputExample(texts=[row[0], row[2]], label=int(0))) # TripletLoss neg
      else:
        continue

print('验证集大小：' + str(valid_shuliang))  

#构造验证集数据 DataLoader
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(valid_samples, name='ipc-clc-valid')


#warmup步数  这个也有很大的随机性
warmup_steps = math.ceil(len(train_samples) * num_epochs / batch_size * 0.15) #15% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


#模型训练
model.fit(train_objectives = [(train_dataloader, train_loss)],
          epochs = num_epochs,
          #optimizer_class = 'transformers.optimization.AdamW',
          optimizer_params = {'correct_bias': False, 'eps': 1e-06, 'lr': 2e-05},
          scheduler = 'WarmupLinear',
          evaluator = evaluator,
          evaluation_steps = 10,
          save_best_model = True,
          weight_decay = weight_decay,
          warmup_steps = warmup_steps,
          max_grad_norm = 1,
          output_path = model_save_path
         )

2021-03-07 15:01:28 - 读入训练数据...
训练集大小：1280
2021-03-07 15:01:28 - 读入验证集数据...
验证集大小：320
2021-03-07 15:01:28 - Warmup-steps: 48


2021-03-07 15:01:35 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 0 after 10 steps:
2021-03-07 15:01:36 - Cosine-Similarity :	Pearson: 0.9003	Spearman: 0.8493
2021-03-07 15:01:36 - Manhattan-Distance:	Pearson: 0.8966	Spearman: 0.8517
2021-03-07 15:01:36 - Euclidean-Distance:	Pearson: 0.8953	Spearman: 0.8513
2021-03-07 15:01:36 - Dot-Product-Similarity:	Pearson: 0.8657	Spearman: 0.8336
2021-03-07 15:01:36 - Save model to output/4-8-training_ipc-to-clc_bert-base-chinese-2021-03-07_15-01-24
2021-03-07 15:01:42 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 0 after 20 steps:
2021-03-07 15:01:43 - Cosine-Similarity :	Pearson: 0.8916	Spearman: 0.8523
2021-03-07 15:01:43 - Manhattan-Distance:	Pearson: 0.9001	Spearman: 0.8553
2021-03-07 15:01:43 - Euclidean-Distance:	Pearson: 0.8992	Spearman: 0.8543
2021-03-07 15:01:43 - Dot-Product-Similarity:	Pearson: 0.8545	Spearman: 0.8334
2021-03-07 15:01:43 - Save model to o

2021-03-07 15:02:07 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 1 after 10 steps:
2021-03-07 15:02:08 - Cosine-Similarity :	Pearson: 0.8536	Spearman: 0.8562
2021-03-07 15:02:08 - Manhattan-Distance:	Pearson: 0.8857	Spearman: 0.8566
2021-03-07 15:02:08 - Euclidean-Distance:	Pearson: 0.8858	Spearman: 0.8568
2021-03-07 15:02:08 - Dot-Product-Similarity:	Pearson: 0.7528	Spearman: 0.7903
2021-03-07 15:02:08 - Save model to output/4-8-training_ipc-to-clc_bert-base-chinese-2021-03-07_15-01-24
2021-03-07 15:02:17 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 1 after 20 steps:
2021-03-07 15:02:18 - Cosine-Similarity :	Pearson: 0.8941	Spearman: 0.8558
2021-03-07 15:02:18 - Manhattan-Distance:	Pearson: 0.8978	Spearman: 0.8565
2021-03-07 15:02:18 - Euclidean-Distance:	Pearson: 0.8980	Spearman: 0.8557
2021-03-07 15:02:18 - Dot-Product-Similarity:	Pearson: 0.8813	Spearman: 0.8531
2021-03-07 15:02:23 - EmbeddingSimila

2021-03-07 15:02:34 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 2 after 10 steps:
2021-03-07 15:02:35 - Cosine-Similarity :	Pearson: 0.9068	Spearman: 0.8587
2021-03-07 15:02:35 - Manhattan-Distance:	Pearson: 0.9138	Spearman: 0.8585
2021-03-07 15:02:35 - Euclidean-Distance:	Pearson: 0.9123	Spearman: 0.8585
2021-03-07 15:02:35 - Dot-Product-Similarity:	Pearson: 0.8903	Spearman: 0.8570
2021-03-07 15:02:35 - Save model to output/4-8-training_ipc-to-clc_bert-base-chinese-2021-03-07_15-01-24
2021-03-07 15:02:44 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 2 after 20 steps:
2021-03-07 15:02:45 - Cosine-Similarity :	Pearson: 0.8953	Spearman: 0.8588
2021-03-07 15:02:45 - Manhattan-Distance:	Pearson: 0.9101	Spearman: 0.8596
2021-03-07 15:02:45 - Euclidean-Distance:	Pearson: 0.9098	Spearman: 0.8594
2021-03-07 15:02:45 - Dot-Product-Similarity:	Pearson: 0.8701	Spearman: 0.8532
2021-03-07 15:02:45 - Save model to o

2021-03-07 15:03:04 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 3 after 10 steps:
2021-03-07 15:03:06 - Cosine-Similarity :	Pearson: 0.8547	Spearman: 0.8568
2021-03-07 15:03:06 - Manhattan-Distance:	Pearson: 0.8894	Spearman: 0.8574
2021-03-07 15:03:06 - Euclidean-Distance:	Pearson: 0.8900	Spearman: 0.8572
2021-03-07 15:03:06 - Dot-Product-Similarity:	Pearson: 0.8403	Spearman: 0.8515
2021-03-07 15:03:10 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 3 after 20 steps:
2021-03-07 15:03:11 - Cosine-Similarity :	Pearson: 0.8833	Spearman: 0.8542
2021-03-07 15:03:11 - Manhattan-Distance:	Pearson: 0.8933	Spearman: 0.8558
2021-03-07 15:03:11 - Euclidean-Distance:	Pearson: 0.8936	Spearman: 0.8556
2021-03-07 15:03:11 - Dot-Product-Similarity:	Pearson: 0.8685	Spearman: 0.8502
2021-03-07 15:03:15 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 3 after 30 steps:
2021-03-07 15:03:

2021-03-07 15:03:27 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 4 after 10 steps:
2021-03-07 15:03:28 - Cosine-Similarity :	Pearson: 0.8718	Spearman: 0.8545
2021-03-07 15:03:28 - Manhattan-Distance:	Pearson: 0.8931	Spearman: 0.8556
2021-03-07 15:03:28 - Euclidean-Distance:	Pearson: 0.8937	Spearman: 0.8559
2021-03-07 15:03:28 - Dot-Product-Similarity:	Pearson: 0.8484	Spearman: 0.8436
2021-03-07 15:03:32 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 4 after 20 steps:
2021-03-07 15:03:33 - Cosine-Similarity :	Pearson: 0.8783	Spearman: 0.8555
2021-03-07 15:03:33 - Manhattan-Distance:	Pearson: 0.8962	Spearman: 0.8553
2021-03-07 15:03:33 - Euclidean-Distance:	Pearson: 0.8970	Spearman: 0.8559
2021-03-07 15:03:33 - Dot-Product-Similarity:	Pearson: 0.8569	Spearman: 0.8449
2021-03-07 15:03:37 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 4 after 30 steps:
2021-03-07 15:03:

2021-03-07 15:03:49 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 5 after 10 steps:
2021-03-07 15:03:50 - Cosine-Similarity :	Pearson: 0.8881	Spearman: 0.8557
2021-03-07 15:03:50 - Manhattan-Distance:	Pearson: 0.9030	Spearman: 0.8563
2021-03-07 15:03:50 - Euclidean-Distance:	Pearson: 0.9034	Spearman: 0.8565
2021-03-07 15:03:50 - Dot-Product-Similarity:	Pearson: 0.8717	Spearman: 0.8497
2021-03-07 15:03:54 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 5 after 20 steps:
2021-03-07 15:03:55 - Cosine-Similarity :	Pearson: 0.8787	Spearman: 0.8549
2021-03-07 15:03:55 - Manhattan-Distance:	Pearson: 0.8999	Spearman: 0.8554
2021-03-07 15:03:55 - Euclidean-Distance:	Pearson: 0.9000	Spearman: 0.8559
2021-03-07 15:03:55 - Dot-Product-Similarity:	Pearson: 0.8589	Spearman: 0.8472
2021-03-07 15:03:59 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 5 after 30 steps:
2021-03-07 15:04:

2021-03-07 15:04:11 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 6 after 10 steps:
2021-03-07 15:04:12 - Cosine-Similarity :	Pearson: 0.8953	Spearman: 0.8576
2021-03-07 15:04:12 - Manhattan-Distance:	Pearson: 0.9088	Spearman: 0.8573
2021-03-07 15:04:12 - Euclidean-Distance:	Pearson: 0.9092	Spearman: 0.8579
2021-03-07 15:04:12 - Dot-Product-Similarity:	Pearson: 0.8793	Spearman: 0.8527
2021-03-07 15:04:16 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 6 after 20 steps:
2021-03-07 15:04:17 - Cosine-Similarity :	Pearson: 0.8960	Spearman: 0.8577
2021-03-07 15:04:17 - Manhattan-Distance:	Pearson: 0.9097	Spearman: 0.8572
2021-03-07 15:04:17 - Euclidean-Distance:	Pearson: 0.9101	Spearman: 0.8580
2021-03-07 15:04:17 - Dot-Product-Similarity:	Pearson: 0.8788	Spearman: 0.8527
2021-03-07 15:04:21 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 6 after 30 steps:
2021-03-07 15:04:

2021-03-07 15:04:32 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 7 after 10 steps:
2021-03-07 15:04:33 - Cosine-Similarity :	Pearson: 0.8958	Spearman: 0.8578
2021-03-07 15:04:33 - Manhattan-Distance:	Pearson: 0.9098	Spearman: 0.8574
2021-03-07 15:04:33 - Euclidean-Distance:	Pearson: 0.9102	Spearman: 0.8578
2021-03-07 15:04:33 - Dot-Product-Similarity:	Pearson: 0.8778	Spearman: 0.8522
2021-03-07 15:04:37 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 7 after 20 steps:
2021-03-07 15:04:39 - Cosine-Similarity :	Pearson: 0.8958	Spearman: 0.8578
2021-03-07 15:04:39 - Manhattan-Distance:	Pearson: 0.9098	Spearman: 0.8574
2021-03-07 15:04:39 - Euclidean-Distance:	Pearson: 0.9102	Spearman: 0.8578
2021-03-07 15:04:39 - Dot-Product-Similarity:	Pearson: 0.8778	Spearman: 0.8522
2021-03-07 15:04:43 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 7 after 30 steps:
2021-03-07 15:04:

In [ ]:
#利用训练好的模型得到测试集结果
model = SentenceTransformer(model_save_path)

evaluator(model, output_path=model_save_path)

2021-03-07 15:04:50 - Load pretrained SentenceTransformer: output/4-8-training_ipc-to-clc_bert-base-chinese-2021-03-07_15-01-24
2021-03-07 15:04:50 - Load SentenceTransformer from folder: output/4-8-training_ipc-to-clc_bert-base-chinese-2021-03-07_15-01-24
2021-03-07 15:04:53 - Use pytorch device: cuda
2021-03-07 15:04:53 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset:
2021-03-07 15:04:54 - Cosine-Similarity :	Pearson: 0.8953	Spearman: 0.8588
2021-03-07 15:04:54 - Manhattan-Distance:	Pearson: 0.9101	Spearman: 0.8596
2021-03-07 15:04:54 - Euclidean-Distance:	Pearson: 0.9098	Spearman: 0.8594
2021-03-07 15:04:54 - Dot-Product-Similarity:	Pearson: 0.8701	Spearman: 0.8532


0.8596020687766998

## 利用预训练语言模型进行验证（这里只需要测试数据，k折交叉验证可以直接改变测试数据）

In [ ]:
#可选，如果不进行上面的训练，则这里不注释，第0个文件中保存有全部1000条数据
#import torch
#from sentence_transformers import SentenceTransformer, util
#from tqdm import tqdm
#kfold = "0"

#读入IPC和CLC测试数据
read_ipc_file = open("data_KFold_1/" + "origin/" + "data_origin_" + kfold + "/ipc_valid.txt", 'r', encoding='utf-8')
read_clc_file = open("data_KFold_1/" + "origin/" + "data_origin_" + kfold + "/clc_valid.txt", 'r', encoding='utf-8')

#存储IPC和CLC的分别类号(key)和匹配字符串(value)
ipc_key_list = []
ipc_value_list = []

clc_key_list = []
clc_value_list = []

for line in read_ipc_file:
    line = line.strip()
    line = line.split(' ')
    assert len(line) == 2, 'IPC.txt文件中只应该有两列，每一列用一个空格隔开！' + str(line[0])
    ipc_key_list.append(str(line[0]))
    ipc_value_list.append(str(line[1]))
    
        
for line in read_clc_file:
    line = line.strip()
    line = line.split(' ')
    assert len(line) == 2, 'CLC.txt文件中只应该有两列，每一列用一个空格隔开！' + str(line[0])
    clc_key_list.append(str(line[0]))
    clc_value_list.append(str(line[1]))
    
assert len(ipc_key_list) == len(clc_key_list), 'IPC.txt文件和CLC.txt文件要一一对应，行数要一致！'


match_clc_key_dict = dict()

for ipc_key_i, target in enumerate(ipc_key_list):
    match_clc_key_dict[str(ipc_key_i)] = []
    for ipc_key_j, every_ipc_key in enumerate(ipc_key_list):
        if every_ipc_key == target:
            match_clc_key_dict[str(ipc_key_i)].append(clc_key_list[ipc_key_j])

#用于计算IPC每一类别包含的数量
ipc_section_number_dict = {
    'A' : 0,
    'B' : 0,
    'C' : 0,
    'D' : 0,
    'E' : 0,
    'F' : 0,
    'G' : 0,
    'H' : 0
}
for ipc_key_i in ipc_key_list:
  ipc_section_number_dict[str(ipc_key_i[0])] += 1



"""
思路：
利用预训练语言模型对IPC和CLC字符串进行编码，得到句子级别的语境向量，
之后对两者进行语义相似度计算（余弦相似度）。

步骤：
1.我们将IPC作为查询字符串，CLC作为待匹配的字符串。

2.一条IPC字符串需要与每一条CLC字符串进行语义相似度计算（余弦相似度）。

3.计算完毕后，取出相似度最高的前n条CLC分类号，作为与当前IPC形成映射的CLC分类号。

4.计算准确率，TopK准确率。
"""

#判断GPU是否可用
device = "cuda" if torch.cuda.is_available() else 'cpu'
#模型路径 （optional:  model_save_path   "bert-base-chinese"   "distiluse-base-multilingual-cased-v2"  "hfl/chinese-bert-wwm"）
model_path = model_save_path

#加载预训练语言模型（可更换,传入模型名称/模型路径）
model = SentenceTransformer(model_path, device=device)

#将CLC查询字符串转化为语境向量，并转化为tensor
clc_corpus_embeddings = model.encode(clc_value_list, convert_to_tensor=True, device=device)

'''
ipdb> len(clc_value_list)
200

ipdb> clc_corpus_embeddings.shape
torch.Size([200, 768])
'''

#可规定TopK准确率
top_k = 10

#存储相似度最高的前n条CLC分类号
clc_top_k_results = dict()

#存储匹配正确的数量(只取匹配度最高的)
correct = 0

#存储TopK中匹配正确的数量
top_k_correct = 0

#存储top2-TopK中匹配正确的数量(包含了topK)
top2_to_topk_correct_dict = {
    '2' : 0,
    '3' : 0,
    '4' : 0,
    '5' : 0,
    '6' : 0,
    '7' : 0,
    '8' : 0,
    '9' : 0,
    '10' : 0
}

#存储每一个IPC类别的正确个数
ipc_section_correct_dict = {
    'A' : 0,
    'B' : 0,
    'C' : 0,
    'D' : 0,
    'E' : 0,
    'F' : 0,
    'G' : 0,
    'H' : 0
}

#存储相似度最高匹配错误的ipc和clc句对
error_match_ipc_clc_pair = []

#存储topk匹配错误的ipc和clc句对
topk_error_match_ipc_clc_pair = []

pbar = tqdm(ipc_value_list)
for ipc_i, ipc_query in enumerate(pbar):
    
    #将IPC查询字符串转化为语境向量，并转化为tensor
    ipc_query_embedding = model.encode(ipc_query, convert_to_tensor=True, device=device)
    '''
    ipdb> ipc_query
    '鞋类;鞋类的特征;鞋类的部件;鞋帮;靴腿;增强件;其他单个鞋部件;鞋帮;靴腿;'

    ipdb> ipc_query_embedding .shape
    torch.Size([768])
    '''
    #计算两者的语义相似度计算（余弦相似度）
    cos_scores = util.pytorch_cos_sim(ipc_query_embedding, clc_corpus_embeddings)[0]
    '''
    ipdb> ipc_query_embedding.shape
    torch.Size([768])

    ipdb> clc_corpus_embeddings.shape
    torch.Size([200, 768])

    ipdb> cos_scores .shape
    torch.Size([200])
    '''
    cos_scores = cos_scores.cpu()
    

    #使用 torch.topk 找到相似度最高的前n条CLC分类号
    top_results = torch.topk(cos_scores, k=top_k)
    '''
    ipdb> top_results 
    torch.return_types.topk(
    values=tensor([0.9510, 0.8601, 0.8227, 0.8227, 0.8227, 0.7801, 0.7547, 0.7545, 0.7543,
            0.7527]),
    indices=tensor([  0,  11, 118, 126, 119,  36,  47,  23,  90,  74]))
    '''
    clc_top_k_results[str(ipc_i)] = []
    to_match_clc_key_list = []
    
    #只有前K个结果
    for score, idx in zip(top_results[0], top_results[1]):
        clc_top_k_results[str(ipc_i)].append((str(clc_key_list[idx]), str(clc_value_list[idx]), score))
        to_match_clc_key_list.append(str(clc_key_list[idx]))
    
    #匹配正确的数量(只取第一个匹配度最高的)
    if clc_top_k_results[str(ipc_i)][0][0] in match_clc_key_dict[str(ipc_i)]:
        correct = correct + 1
        ipc_section_correct_dict[str(ipc_key_list[ipc_i][0])] += 1
    else:
        error_match_ipc_clc_pair.append(str(ipc_key_list[ipc_i]) + '  ' + str(ipc_value_list[ipc_i]) + \
                                        '\t|||\t' + str(clc_key_list[ipc_i]) + '  ' + \
                                        str(clc_value_list[ipc_i]) + '\t|||\t' + \
                                        str(clc_top_k_results[str(ipc_i)][0][0]) + '  ' + \
                                        str(clc_top_k_results[str(ipc_i)][0][1]))
    
    #TopK中匹配正确的数量(10个的)
    if len(match_clc_key_dict[str(ipc_i)]) == 1:
      assert match_clc_key_dict[str(ipc_i)][0] == str(clc_key_list[ipc_i]), 'TopK匹配有问题！--1'
      if str(clc_key_list[ipc_i]) in to_match_clc_key_list:
        top_k_correct = top_k_correct + 1
      else:
        topk_error_match_ipc_clc_pair.append(str(ipc_key_list[ipc_i]) + '  ' + str(ipc_value_list[ipc_i]) + \
                                          '\t|||\t' + str(clc_key_list[ipc_i]) + '  ' + \
                                          str(clc_value_list[ipc_i]) + '\t|||\t' + \
                                          str(clc_top_k_results[str(ipc_i)][0][0]) + '  ' + \
                                          str(clc_top_k_results[str(ipc_i)][0][1]))
    elif len(match_clc_key_dict[str(ipc_i)]) > 1:
      for each_clc_key_1 in match_clc_key_dict[str(ipc_i)]:
        if str(each_clc_key_1) in to_match_clc_key_list:
          top_k_correct = top_k_correct + 1
          break
        else:
          topk_error_match_ipc_clc_pair.append(str(ipc_key_list[ipc_i]) + '  ' + str(ipc_value_list[ipc_i]) + \
                                            '\t|||\t' + str(clc_key_list[ipc_i]) + '  ' + \
                                            str(clc_value_list[ipc_i]) + '\t|||\t' + \
                                            str(clc_top_k_results[str(ipc_i)][0][0]) + '  ' + \
                                            str(clc_top_k_results[str(ipc_i)][0][1]))
    else:
      print('TopK匹配有问题！--2')

    
    #计算tok2-topK每一个的准确率
    for topk_index in range(1, top_k):
      for each_clc_key_1 in match_clc_key_dict[str(ipc_i)]:
        if str(each_clc_key_1) in to_match_clc_key_list[:topk_index+1]:
          top2_to_topk_correct_dict[str(topk_index+1)] += 1
          break

    pbar.set_description("Processing %s" % str(ipc_i+1))
    
    
#计算准确率，TopK准确率
ipc_count_number = len(ipc_key_list)
acc = correct/ipc_count_number
top_k_acc = top_k_correct/ipc_count_number
print('\nFinal acc: {:.2f}% .'.format(acc*100))
print('\nFinal top_{}_acc: {:.2f}% .'.format(top_k, top_k_acc*100))
print('\n--------------------\n')
for topk_name, top_correct_num in top2_to_topk_correct_dict.items():
  print('\nFinal top_{}_acc: {:.2f}% .'.format(topk_name, (top_correct_num/ipc_count_number)*100))

2021-03-07 15:04:54 - Load pretrained SentenceTransformer: output/4-8-training_ipc-to-clc_bert-base-chinese-2021-03-07_15-01-24
2021-03-07 15:04:54 - Load SentenceTransformer from folder: output/4-8-training_ipc-to-clc_bert-base-chinese-2021-03-07_15-01-24


  0%|          | 0/200 [00:00<?, ?it/s]

Processing 1:   0%|          | 0/200 [00:00<?, ?it/s]

Processing 2:   1%|          | 2/200 [00:00<00:18, 10.46it/s]

Processing 3:   2%|▏         | 3/200 [00:00<00:19, 10.07it/s]

...

Processing 197:  98%|█████████▊| 197/200 [00:28<00:00,  5.33it/s]

Processing 198:  99%|█████████▉| 198/200 [00:29<00:00,  5.74it/s]

Processing 199: 100%|█████████▉| 199/200 [00:29<00:00,  5.83it/s]

Processing 200: 100%|██████████| 200/200 [00:29<00:00,  6.82it/s]



Final acc: 83.50% .

Final top_10_acc: 98.00% .

--------------------


Final top_2_acc: 92.00% .

Final top_3_acc: 94.50% .

Final top_4_acc: 96.00% .

Final top_5_acc: 97.00% .

Final top_6_acc: 97.50% .

Final top_7_acc: 98.00% .

Final top_8_acc: 98.00% .

Final top_9_acc: 98.00% .

Final top_10_acc: 98.00% .


In [ ]:
#结果统计
print('总数：{}\ntop1 正确个数：{}\ntop1 错误个数：{}\n'.format(ipc_count_number, correct, len(error_match_ipc_clc_pair)))

print('总数：{}\ntop10 正确个数：{}\ntop10 错误个数：{}\n'.format(ipc_count_number, top_k_correct, len(topk_error_match_ipc_clc_pair)))


print('\n---------------------------------\n')
for topk_name, top_correct_num in top2_to_topk_correct_dict.items():
  print('总数：{}\ntop{} 正确个数：{}\ntop{} 错误个数：{}\n'.format(ipc_count_number, topk_name, top_correct_num, topk_name, ipc_count_number-top_correct_num))

总数：200
top1 正确个数：167
top1 错误个数：33

总数：200
top10 正确个数：196
top10 错误个数：4


---------------------------------

总数：200
top2 正确个数：184
top2 错误个数：16

总数：200
top3 正确个数：189
top3 错误个数：11

总数：200
top4 正确个数：192
top4 错误个数：8

总数：200
top5 正确个数：194
top5 错误个数：6

总数：200
top6 正确个数：195
top6 错误个数：5

总数：200
top7 正确个数：196
top7 错误个数：4

总数：200
top8 正确个数：196
top8 错误个数：4

总数：200
top9 正确个数：196
top9 错误个数：4

总数：200
top10 正确个数：196
top10 错误个数：4



In [ ]:
#计算每一个IPC类别的正确个数和正确率（只有top1的）
for sec_i, sec_cor in ipc_section_correct_dict.items():
  print('类别：{}，总数为：{}，正确个数为：{}，错误个数为：{}，正确率为：{:.2f}%\n'.format(sec_i, ipc_section_number_dict[sec_i], sec_cor, ipc_section_number_dict[sec_i]-sec_cor, (sec_cor/ipc_section_number_dict[sec_i])*100))

类别：A，总数为：18，正确个数为：16，错误个数为：2，正确率为：88.89%

类别：B，总数为：40，正确个数为：29，错误个数为：11，正确率为：72.50%

类别：C，总数为：28，正确个数为：27，错误个数为：1，正确率为：96.43%

类别：D，总数为：19，正确个数为：18，错误个数为：1，正确率为：94.74%

类别：E，总数为：18，正确个数为：16，错误个数为：2，正确率为：88.89%

类别：F，总数为：27，正确个数为：20，错误个数为：7，正确率为：74.07%

类别：G，总数为：24，正确个数为：18，错误个数为：6，正确率为：75.00%

类别：H，总数为：26，正确个数为：23，错误个数为：3，正确率为：88.46%



In [ ]:
#可选，如果不进行上面的训练，则这里不注释
loss_type = 'Transformer_Encoder'
#kfold = model_path
#num_epochs = 'no_train'

#相似度最高的错误匹配（topk=1）
f_write = open('error_log/' + loss_type + '_log/' + 'data_origin_' + kfold + '/' + str(num_epochs) + '-' + str(round(acc*100, 2)) + '%-' + 'top1_error_text_log.txt', 'w', encoding='utf-8')

for k_v in error_match_ipc_clc_pair:
  f_write.write(str(k_v) + '\n')

f_write.close()

In [ ]:
#topk的错误匹配结果
f_write = open('error_log/' + loss_type + '_log/' + 'data_origin_' + kfold + '/' + str(num_epochs) + '-' + str(round(acc*100, 2)) + '%-' + 'topk_error_text_log.txt', 'w', encoding='utf-8')

for k_v in topk_error_match_ipc_clc_pair:
  f_write.write(str(k_v) + '\n')

f_write.close()

In [ ]:
#topk的匹配结果
f_write = open('error_log/' + loss_type + '_log/' + 'data_origin_' + kfold + '/' + str(num_epochs) + '-' + str(round(acc*100, 2)) + '%-' + 'topk_error_score_log.txt', 'w', encoding='utf-8')

for k_index, v_list_tuple in clc_top_k_results.items():
  for each_tuple in v_list_tuple:
    f_write.write(ipc_key_list[int(k_index)] + '----' + each_tuple[0] + each_tuple[1] + str(each_tuple[2].item()) + '\n')

  f_write.write('------------------------------------------------------------------------------' + '\n')

f_write.close()